In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# built in python modules
import datetime
import os
import inspect
import sys

# python add-ons
import numpy as np
import pandas as pd
import xarray as xr
import netCDF4
import wrf

# # Import the pvlib module
if sys.platform == 'linux':
    sys.path.append('/home/jsward/Documents/01_Research/01_Renewable_Analysis/WRF/pvlib-python')
import pvlib
from pvlib.wrfcast import WRF

In [ ]:
# Find the absolute file path to your pvlib installation
pvlib_abspath = os.path.dirname(os.path.abspath(inspect.getfile(pvlib)))

# absolute path to WRF data file
datapath = os.path.join(pvlib_abspath, 'data', 'wrfout_d01_2011-01-24_01:00:00')

# Read in the wrfout file using the netCDF4.Dataset method (I think you can also do this with an xarray method)
netcdf_data = netCDF4.Dataset(datapath)

In [48]:
# Create an xarray.Dataset from the wrf qurery_variables.
query_variables = [
            'Times',
            'T2',
            'U10',
            'V10',
            'SWDDNI',
            'SWDDIF'
            ]
first = True
for key in query_variables:
    var = wrf.getvar(netcdf_data, key, timeidx=wrf.ALL_TIMES)
    if first:
        solar_data = var
        first = False
    else:
        solar_data = xr.merge([solar_data, var])

variables = {
            'times': 'times',
            'XLAT': 'lat',
            'XLONG': 'lon',
            'T2': 'temp_air',
            'U10': 'wind_speed_u',
            'V10': 'wind_speed_v',
            'SWDDNI': 'dni',
            'SWDDIF': 'dhi' 
            }
solar_data = xr.Dataset.rename(solar_data, variables)
solar_data = xr.Dataset.reset_coords(solar_data, ['XTIME'], drop=True)
times = solar_data.times
solar_data = xr.Dataset.set_coords(solar_data, ['times'])
solar_data = xr.Dataset.reset_coords(solar_data, ['times'], drop=True)
ntimes = solar_data.sizes['Time']
nlat = solar_data.sizes['south_north']
nlon = solar_data.sizes['west_east']
solar_data

<xarray.Dataset>
Dimensions:       (Time: 24, south_north: 191, west_east: 191)
Coordinates:
  * Time          (Time) datetime64[ns] 2011-01-24T01:00:00 ... 2011-01-25
    lon           (south_north, west_east) float32 -95.43335 ... -64.11832
    lat           (south_north, west_east) float32 30.952759 ... 47.507103
Dimensions without coordinates: south_north, west_east
Data variables:
    temp_air      (Time, south_north, west_east) float32 287.62427 ... 264.03226
    wind_speed_u  (Time, south_north, west_east) float32 0.31499797 ... 17.972435
    wind_speed_v  (Time, south_north, west_east) float32 1.1208413 ... -1.0850167
    dni           (Time, south_north, west_east) float32 0.0 0.0 0.0 ... 0.0 0.0
    dhi           (Time, south_north, west_east) float32 0.0 0.0 0.0 ... 0.0 0.0

In [49]:
# Explore how the WRF forecast model behaves
fm = WRF()
wind_speed = fm.uv_to_speed(solar_data)
temp_air = fm.kelvin_to_celsius(solar_data['temp_air'])

In [50]:
# Convert xarray Datasets to a pandas DataFrames
solar_data = solar_data.to_dataframe()
times = times.to_dataframe()
solar_data

lon        lat    temp_air  \
Time                south_north west_east                                     
2011-01-24 01:00:00 0           0         -95.433350  30.952759  287.624268   
                                1         -95.308044  30.950832  287.592499   
                                2         -95.182739  30.948757  287.163452   
                                3         -95.057465  30.946545  287.356171   
                                4         -94.932190  30.944153  287.152344   
...                                              ...        ...         ...   
2011-01-25 00:00:00 190         186       -64.714050  47.657688  263.186615   
                                187       -64.564850  47.620308  262.584229   
                                188       -64.415833  47.582741  263.205566   
                                189       -64.266968  47.545010  263.703674   
                                190       -64.118317  47.507103  264.032257   

                                           wind_speed_u  wind_speed_v  dni  \
Time                south_north west_east                                    
2011-01-24 01:00:00 0           0              0.314998      1.120841  0.0   
                                1             -0.087709      1.663511  0.0   
                                2             -0.358935      1.999503  0.0   
                                3             -0.632014      2.513421  0.0   
                                4             -0.803844      2.692272  0.0   
...                                                 ...           ...  ...   
2011-01-25 00:00:00 190         186           15.262465     -0.319409  0.0   
                                187           15.590679     -0.548230  0.0   
                                188           16.902138     -0.615797  0.0   
                                189           17.802485     -0.596332  0.0   
                                190           17.972435     -1.085017  0.0   

                                           dhi  
Time                south_north west_east       
2011-01-24 01:00:00 0           0          0.0  
                                1          0.0  
                                2          0.0  
                                3          0.0  
                                4          0.0  
...                                        ...  
2011-01-25 00:00:00 190         186        0.0  
                                187        0.0  
                                188        0.0  
                                189        0.0  
                                190        0.0  

[875544 rows x 7 columns]

In [ ]:
# Run the solar position algorithm for time, lat, and lon indices, and concatonate them into a single DataFrame
# This takes PROHIBITIVELY long time...
numerical_time_indices = range(0, ntimes)
lat_indices = range(0, nlat)
lon_indices = range(0, nlon)
first = True
for num_time_idx in numerical_time_indices:
    time = times.index.get_level_values('Time')[num_time_idx]
    print(f'Time Index: {time}')
    for lat_idx in lat_indices:
        for lon_idx in lon_indices:
            
#             print(f'Time Index: {time}')
#             print(f'\tLatitude index: {lat_idx}')
#             print(f'\t\tLongitude index: {lon_idx}')
            
            solpos_new = pvlib.solarposition.spa_xarray_python(time, 
                                        solar_data['lat'].loc[time, lat_idx, lon_idx], 
                                        solar_data['lon'].loc[time, lat_idx, lon_idx], lat_idx, lon_idx)
            
            if first:
                solpos = solpos_new
                first = False
            else:
                solpos = pd.concat([solpos, solpos_new])

Time Index: 2011-01-24 01:00:00
Time Index: 2011-01-24 02:00:00
Time Index: 2011-01-24 03:00:00
Time Index: 2011-01-24 04:00:00
Time Index: 2011-01-24 05:00:00
Time Index: 2011-01-24 06:00:00
Time Index: 2011-01-24 07:00:00
Time Index: 2011-01-24 08:00:00
Time Index: 2011-01-24 09:00:00


In [ ]:
solpos

In [ ]:
# This kills the kernel...
solar_data['zenith'] = solpos['zenith']
# solar_data = solar_data.to_xarray()

In [ ]:
 ghi = fm.dni_and_dhi_to_ghi(solar_data['dni'], solar_data['dhi'], solar_data['zenith'])

In [ ]:
solar_data['ghi'] = ghi

In [ ]:
solar_data